# M02 Importing Persuasion as OHCO

DS 5001 Text as Data

## Purpose

We import a text as an **OHCO** using the  **Clip, Chunk, and Split pattern**.

We demonstrate how to tokenize a raw text and map an OHCO onto the resulting dataframe of tokens.

This goes beyond what we did last week in the _First Foray_ notebook. We capture the chapter, paragraph, and sentence structure of the text.

## Set Up

## Import libraries

In [1]:
import pandas as pd

### Import Config

In [2]:
data_home = "../input"
output_dir = "../working"

In [3]:
data_home, output_dir

('../input', '../working')

### Parameters

In [4]:
pg_id = 105
pg_title = "austen-persuasion"

In [5]:
text_file = f"{data_home}/gutenberg/pg{pg_id}.txt"
csv_file  = f"{output_dir}/{pg_title}.csv" # The file we will create

In [6]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

## Import file into a dataframe

In [7]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

In [8]:
LINES.sample(20)

,line_str
line_num,
4064,"handsome, and gave her more pleasure than almo..."
8373,*** END OF THIS PROJECT GUTENBERG EBOOK PERSUA...
1880,also to wait on her for a few minutes if not i...
3614,"""Captain Benwick, would not it be better for C..."
2371,when a certain Charles Hayter returned among t...
1261,every direction. Oh! could the originals of t...
6407,"""Now, how I do wish I understood you! How I d..."
2409,"now, whether the more gentle or the more livel..."
5190,this picture excited. For a few moments her i...


## Extract Title 

In [9]:
title = LINES.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')

In [10]:
print(title)

Persuasion, by Jane Austen


## Clip the Cruft

In [11]:
clip_pats = [
    r"\*\*\*\s*START OF (?:THE|THIS) PROJECT",
    r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"
]

In [12]:
pat_a = LINES.line_str.str.match(clip_pats[0])
pat_b = LINES.line_str.str.match(clip_pats[1])

In [13]:
line_a = LINES.loc[pat_a].index[0] + 1
line_b = LINES.loc[pat_b].index[0] - 1

In [14]:
line_a, line_b

(np.int64(19), np.int64(8372))

In [15]:
LINES = LINES.loc[line_a : line_b]

In [16]:
LINES.head(10)

,line_str
line_num,
19,
20,
21,
22,
23,Produced by Sharon Partridge and Martin Ward. ...
24,by Al Haines.
25,
26,
27,


In [17]:
LINES.tail(10)

,line_str
line_num,
8363,
8364,
8365,
8366,
8367,
8368,
8369,
8370,
8371,End of the Project Gutenberg EBook of Persuasi...


## Chunk by Chapter

### Find all chapter headers

The regex will depend on the source text. You need to investigate the source text to figure this out.

In [18]:
chap_pat = r"^\s*(?:chapter|letter)\s+\d+"

In [19]:
chap_lines = LINES.line_str.str.match(chap_pat, case=False) # Returns a truth vector

In [20]:
LINES.loc[chap_lines] # Use as filter for dataframe

,line_str
line_num,
47,Chapter 1
306,Chapter 2
500,Chapter 3
786,Chapter 4
959,Chapter 5
1297,Chapter 6
1657,Chapter 7
1992,Chapter 8
2346,Chapter 9


### Assign numbers to chapters

In [21]:
LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]

In [22]:
LINES.loc[chap_lines]

,line_str,chap_num
line_num,,
47,Chapter 1,1.0
306,Chapter 2,2.0
500,Chapter 3,3.0
786,Chapter 4,4.0
959,Chapter 5,5.0
1297,Chapter 6,6.0
1657,Chapter 7,7.0
1992,Chapter 8,8.0
2346,Chapter 9,9.0


Notice that all lines that are not chapter headers have no chapter number assigned to them.

In [23]:
LINES.sample(10)

,line_str,chap_num
line_num,,
7787,lives. I believe you equal to every important...,NaN
1905,"Soon, however, she began to reason with hersel...",NaN
6365,reflection shewed her the mistake she had been...,NaN
8177,strong sense of duty is no bad part of a woman...,NaN
6819,all hours; but I need not be particular on thi...,NaN
3018,,NaN
7578,"be outwardly composed, and feel herself plunge...",NaN
3637,"heaven's name, is to be done next?""",NaN
1677,entirely aside; but she could not hear of her ...,NaN


### Forward-fill chapter numbers to following text lines

`ffill()` will replace null values with the previous non-null value.

In [24]:
LINES.chap_num = LINES.chap_num.ffill()

In [25]:
LINES.sample(10)

,line_str,chap_num
line_num,,
1378,"receiving hints to exert it, beyond what was p...",6.0
6286,Prettier musings of high-wrought love and eter...,21.0
3191,of course that they should dine with them.,11.0
5303,convenient to me to dine there to-morrow. I a...,18.0
4310,,14.0
5431,,18.0
5297,Henrietta did not remain at Lyme as long as Lo...,18.0
3212,"rare species of wood, excellently worked up, a...",11.0
1062,"such an absurd suspicion should occur to her, ...",5.0


Notice that the lines taht precede our first chapter have no chapters, which is what we want. We need to decide whether to keep these lines as textual front matter or to dispose of them.

In [26]:
LINES.head(20)

,line_str,chap_num
line_num,,
19,,NaN
20,,NaN
21,,NaN
22,,NaN
23,Produced by Sharon Partridge and Martin Ward. ...,NaN
24,by Al Haines.,NaN
25,,NaN
26,,NaN
27,,NaN


### Clean up

In [27]:
LINES = LINES.dropna(subset=['chap_num']) # Remove everything before Chapter 1
# LINES = LINES.loc[~LINES.chap_num.isna()] # Remove everything before Chapter 1 (alternate method)
LINES = LINES.loc[~chap_lines] # Remove chapter heading lines; their work is done
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

In [28]:
LINES.sample(10)

,line_str,chap_num
line_num,,
8034,That neither Harville nor his wife entertained...,23
4674,,15
1436,Nobody doubts her right to have precedence of ...,6
3189,as a excuse; but they seemed almost hurt that ...,11
3955,"desirous to be gone, she could not quit the Ma...",13
2293,"myself at Deal, when the Admiral (Captain Crof...",8
1326,"The Mr Musgroves had their own game to guard, ...",6
5972,"understanding, but Benwick is something more. ...",20
3968,being possible for her to go to the Hall she h...,13


### Group lines into chapters

In [29]:
OHCO[:1]

['chap_num']

In [30]:
# Make big string for each chapter
CHAPS = LINES.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

In [31]:
CHAPS.head(10)

,chap_str
chap_num,
1,"\n\nSir Walter Elliot, of Kellynch Hall, in So..."
2,"\n\nMr Shepherd, a civil, cautious lawyer, who..."
3,"\n\n""I must take leave to observe, Sir Walter,..."
4,"\n\nHe was not Mr Wentworth, the former curate..."
5,\n\nOn the morning appointed for Admiral and M...
6,\n\nAnne had not wanted this visit to Uppercro...
7,"\n\nA very few days more, and Captain Wentwort..."
8,\n\nFrom this time Captain Wentworth and Anne ...
9,\n\nCaptain Wentworth was come to Kellynch as ...


In [32]:
CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

In [33]:
CHAPS

,chap_str
chap_num,
1,"Sir Walter Elliot, of Kellynch Hall, in Somers..."
2,"Mr Shepherd, a civil, cautious lawyer, who, wh..."
3,"""I must take leave to observe, Sir Walter,"" sa..."
4,"He was not Mr Wentworth, the former curate of ..."
5,On the morning appointed for Admiral and Mrs C...
6,"Anne had not wanted this visit to Uppercross, ..."
7,"A very few days more, and Captain Wentworth wa..."
8,From this time Captain Wentworth and Anne Elli...
9,Captain Wentworth was come to Kellynch as to a...


So, now we have our text grouped by chapters.

In [34]:
CHAPS.to_csv(f"{output_dir}/pg105-CHAPS.csv", index=True)

## Split chapters into paragraphs 

We use Pandas' convenient `.split()` method with `expand=True`, followed by `.stack()`.
Note that this creates zero-based indexes.

In [35]:
para_pat = r'\n\n+'

In [36]:
# CHAPS['chap_str'].str.split(para_pat, expand=True).head()

In [37]:
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[:2]

In [38]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

In [39]:
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
PARAS['para_str'] = PARAS['para_str'].str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [40]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

In [41]:
PARAS.to_csv(f"{output_dir}/pg105-PARAS.csv", index=True)

## Split paragraphs into sentences

In [42]:
# sent_pat = r'[.?!;:"]+'
sent_pat = r'[.?!;:]+'
SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [43]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS

In [44]:
SENTS.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
                  1         there he found occupation for an idle hour, an...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations, arising from d...
                  4         and there, if every other leaf were powerless,...

In [45]:
SENTS.sample(10)

,,,sent_str
chap_num,para_num,sent_num,
23,81,1,""" he cried, ""you would"
5,16,0,Anne had done
23,62,9,and that I had no right to be trying whether I...
21,79,2,""""
22,16,0,"""And mine,"" added Sir Walter"
10,13,1,"an indifferent house, standing low, and hemmed..."
5,42,0,"""Oh"
10,7,7,but my sister makes nothing of it
3,31,3,"Came there about the year ---5, I take it"


## Split sentences into tokens

In [46]:
token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')

In [47]:
TOKENS.index.names = OHCO[:4]

In [48]:
TOKENS

token_str
chap_num para_num sent_num token_num            
1        0        0        0                 Sir
                           1              Walter
                           2              Elliot
                           3                  of
                           4            Kellynch
...                                          ...
24       13       0        6                  of
                           7          Persuasion
                           8                  by
                           9                Jane
                           10             Austen

[85014 rows x 1 columns]

## Extract Vocabulary

In [49]:
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str  term_str
chap_num para_num sent_num token_num                    
1        0        0        0               Sir       sir
                           1            Walter    walter
                           2            Elliot    elliot
                           3                of        of
                           4          Kellynch  kellynch

In [50]:
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [51]:
VOCAB

,term_str,n
term_id,,
0,the,3330
1,to,2808
2,and,2800
3,of,2572
4,a,1595
...,...,...
5755,acquisition,1
5756,spoiled,1
5757,render,1


In [52]:
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB.head()

,term_str,n,p
term_id,,,
0,the,3330,0.039170
1,to,2808,0.033030
2,and,2800,0.032936
3,of,2572,0.030254
4,a,1595,0.018762


## Gathering by Content Object

In [53]:
def gather(ohco_level):
    level_name = OHCO[ohco_level-1].split('_')[0]
    df = TOKENS.groupby(OHCO[:ohco_level])\
        .token_str.apply(lambda x: x.str.cat(sep=' '))\
        .to_frame(f"{level_name}_str")
    return df

In [54]:
gather(1)

,chap_str
chap_num,
1,Sir Walter Elliot of Kellynch Hall in Somerset...
2,Mr Shepherd a civil cautious lawyer who whatev...
3,"""I must take leave to observe Sir Walter "" sai..."
4,He was not Mr Wentworth the former curate of M...
5,On the morning appointed for Admiral and Mrs C...
6,Anne had not wanted this visit to Uppercross t...
7,A very few days more and Captain Wentworth was...
8,From this time Captain Wentworth and Anne Elli...
9,Captain Wentworth was come to Kellynch as to a...


In [55]:
gather(2)

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot of Kellynch Hall in Somerset...
         1                                  "ELLIOT OF KELLYNCH HALL
         2         "Walter Elliot born March 1 1760 married July ...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...
...                                                              ...
24       9         Anne satisfied at a very early period of Lady ...
         10        Her recent good offices by Anne had been enoug...
         11        Mrs Smith s enjoyments were not spoiled by thi...
         12                                                    Finis
         13        End of the Project Gutenberg EBook of Persuasi...

[1008 rows x 1 columns]

In [56]:
gather(3)

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot of Kellynch Hall in Somerset...
                  1         there he found occupation for an idle hour and...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations arising from do...
                  4         and there if every other leaf were powerless h...
...                                                                       ...
24       11       4         Anne was tenderness itself and she had the ful...
                  5         His profession was all that could ever make he...
                  6         She gloried in being a sailor s wife but she m...
         12       0                                                     Finis
         13       0         End of the Project Gutenberg EBook of Persuasi...

[5612 rows x 1 columns]

## Save work to CSV

This is important -- will be used for homework.

In [57]:
TOKENS.to_csv(csv_file)